In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
print('DONE')


DONE


In [2]:
import cv2
import os
import random
from tqdm import tqdm

# Define paths for validation folders
val_violence_path = "/kaggle/input/rwf2000/RWF-2000/val/Fight"
val_nonviolence_path = "/kaggle/input/rwf2000/RWF-2000/val/NonFight"

# Output paths for extracted frames
output_paths = {
    "train_violence": "/kaggle/working/extracted_frames/train/violence",
    "train_nonviolence": "/kaggle/working/extracted_frames/train/nonviolence",
    "val_violence": "/kaggle/working/extracted_frames/val/violence",
    "val_nonviolence": "/kaggle/working/extracted_frames/val/nonviolence"
}

# Create output directories if they don't exist
for path in output_paths.values():
    os.makedirs(path, exist_ok=True)

# Function to extract frames at a rate of 3 frames per second
def extract_frames(video_path, output_folder, target_fps=3):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps / target_fps)
    frame_count, saved_frame_count = 0, 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frame_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_frame{saved_frame_count}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), frame)
            saved_frame_count += 1
        frame_count += 1
    cap.release()

# Function to process videos, splitting 10% for validation if needed
def process_videos(video_paths, train_output, val_output, split_for_val=False, target_fps=3):
    for folder in video_paths:
        video_files = [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        
        if split_for_val:
            random.shuffle(video_files)
            val_count = max(1, int(len(video_files) * 0.2))  # Take 10% for validation
            val_videos, train_videos = video_files[:val_count], video_files[val_count:]
        else:
            val_videos, train_videos = [], video_files  # All to training if no split
        
        # Extract frames from validation videos
        for video_file in tqdm(val_videos, desc=f"Extracting validation frames from {folder}"):
            extract_frames(os.path.join(folder, video_file), val_output, target_fps=target_fps)

        # Extract frames from training videos
        for video_file in tqdm(train_videos, desc=f"Extracting training frames from {folder}"):
            extract_frames(os.path.join(folder, video_file), train_output, target_fps=target_fps)

# Process violence and nonviolence folders with a 10% split for validation
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)

# Process all files from RWF-2000 train folders as training data (no validation split)
process_videos(
    ["/kaggle/input/rwf2000/RWF-2000/train/Fight"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/rwf2000/RWF-2000/train/NonFight"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)


print("Frame extraction completed.")


Extracting validation frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence: 100%|██████████| 200/200 [00:49<00:00,  4.04it/s]
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence: 100%|██████████| 800/800 [02:16<00:00,  5.87it/s]
Extracting validation frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence: 100%|██████████| 200/200 [00:15<00:00, 12.93it/s]
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence:  82%|████████▏ | 653/800 [01:04<00:11, 12.87it/s][h264 @ 0x56f729ed60c0] mb_type 104 in P slice too large at 98 31
[h264 @ 0x56f729ed60c0] error while decoding MB 98 31
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence: 100%|██████████| 800/800 [01:20<00:00,  9.99it/s]
Extracting va

Frame extraction completed.


In [3]:
import os

# Specify the path to the main directory
main_dir = '/kaggle/working/extracted_frames'

# Dictionary to store file counts
file_counts = {}

# Walk through each subdirectory and count files
for root, dirs, files in os.walk(main_dir):
    # Only count files, skip directories
    file_counts[root] = len(files)

# Print the file counts in each folder
for folder, count in file_counts.items():
    print(f"{folder}: {count} files")


/kaggle/working/extracted_frames: 0 files
/kaggle/working/extracted_frames/train: 0 files
/kaggle/working/extracted_frames/train/violence: 22621 files
/kaggle/working/extracted_frames/train/nonviolence: 23097 files
/kaggle/working/extracted_frames/val: 0 files
/kaggle/working/extracted_frames/val/violence: 6706 files
/kaggle/working/extracted_frames/val/nonviolence: 5689 files


In [4]:
# Paths to augmented frames for training and validation
train_dir = '/kaggle/working/extracted_frames/train'
val_dir = '/kaggle/working/extracted_frames/val'
# Image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16


"""
   ,
    rotation_range=5,              # Small rotation, up to 5 degrees
    width_shift_range=0.05,        # Small horizontal shift, up to 5% of width
    height_shift_range=0.05,       # Small vertical shift, up to 5% of height
    zoom_range=0.1,                # Slight zoom variation, up to 10%
    fill_mode='nearest' 
    """
# Data augmentation and generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'  
)


val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= True ,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
)

Found 45718 images belonging to 2 classes.
Found 12395 images belonging to 2 classes.


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define the MobileNetV3Small model
input_tensor = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model = MobileNetV3Small(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')



base_model.trainable = True  # Initially freeze the entire base model
"""# Unfreeze the top 10 layers
for layer in base_model.layers[-30:]:
    layer.trainable = True
    , kernel_regularizer=l2(0.01)
"""
    

# Add custom layers on top of MobileNetV3Small
x = base_model(input_tensor)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)  # Add Dropout

x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)  # Add Dropout

output = Dense(1, activation='sigmoid')(x)

# Build the model
model = models.Model(inputs=input_tensor, outputs=output)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobilenetV3small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │       590,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_58 (Activation)      │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_59 (Activation)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,055,281 (7.84 MB)

 Trainable params: 2,043,169 (7.79 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint 

model.compile(optimizer=Adam(learning_rate=1e-4),  # Lower LR for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks for adaptive learning rate and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)  
#early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Path to save the best model
    monitor='val_loss',       # Metric to monitor for improvement
    save_best_only=True,      # Save only if the model improves
    mode='min',               # Mode: 'min' for loss, 'max' for accuracy
    verbose=1                 # Print messages when saving
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[checkpoint,reduce_lr]
)

Epoch 1/30
1948/2858 ━━━━━━━━━━━━━━━━━━━━ 3:18 218ms/step - accuracy: 0.7333 - loss: 0.5169

I0000 00:00:1732936393.125246    8190 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion', 36 bytes spill stores, 36 bytes spill loads



2858/2858 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.7547 - loss: 0.4849
Epoch 1: val_loss improved from inf to 2.30799, saving model to best_model.keras
2858/2858 ━━━━━━━━━━━━━━━━━━━━ 705s 226ms/step - accuracy: 0.7547 - loss: 0.4849 - val_accuracy: 0.4590 - val_loss: 2.3080 - learning_rate: 1.0000e-04
Epoch 2/30
2856/2858 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.8818 - loss: 0.2660
Epoch 2: val_loss improved from 2.30799 to 1.59476, saving model to best_model.keras
2858/2858 ━━━━━━━━━━━━━━━━━━━━ 621s 217ms/step - accuracy: 0.8818 - loss: 0.2659 - val_accuracy: 0.5803 - val_loss: 1.5948 - learning_rate: 1.0000e-04
Epoch 3/30
2856/2858 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.9162 - loss: 0.1971
Epoch 3: val_loss improved from 1.59476 to 0.87873, saving model to best_model.keras
2858/2858 ━━━━━━━━━━━━━━━━━━━━ 620s 216ms/step - accuracy: 0.9162 - loss: 0.1971 - val_accuracy: 0.6623 - val_loss: 0.8787 - learning_rate: 1.0000e-04
Epoch 4/30
2858/2858 ━━━━━━━━━━━━━━━━

In [14]:
from sklearn.metrics import classification_report
import numpy as np

# Step 1: Evaluate the model using the validation generator
loss, accuracy = model.evaluate(val_generator, verbose=1)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Step 2: Generate predictions
predictions = model.predict(val_generator, verbose=1)          
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)  # Binary classification: Threshold at 0.5

# Step 3: Get true labels from the generator
true_classes = val_generator.classes  # Assuming this provides the true labels for each sample
class_labels = ['Class 0', 'Class 1']  # Replace with actual class names if available

# Step 4: Generate classification report
print("\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


775/775 ━━━━━━━━━━━━━━━━━━━━ 30s 39ms/step - accuracy: 0.8730 - loss: 1.0437
Validation Loss: 0.9060072302818298
Validation Accuracy: 0.8943929076194763
775/775 ━━━━━━━━━━━━━━━━━━━━ 35s 41ms/step

Classification Report:

              precision    recall  f1-score   support

     Class 0       0.90      0.86      0.88      5689
     Class 1       0.89      0.92      0.90      6706

    accuracy                           0.89     12395
   macro avg       0.90      0.89      0.89     12395
weighted avg       0.89      0.89      0.89     12395



In [15]:
# Save the model in .keras format
model.save("movileV3_89.keras")


In [17]:
pip install tf2onnx


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import tf2onnx
import tensorflow as tf

# Assuming 'model' is your trained Keras or TensorFlow model

# Define the path to save the ONNX model
onnx_model_path = "movileV3_89.onnx"

# Convert the model to ONNX format
spec = (tf.TensorSpec(model.inputs[0].shape, tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)

# Save the ONNX model to the specified path
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved in ONNX format at {onnx_model_path}")


Model saved in ONNX format at movileV3_89.onnx
